In [ ]:
import pandas as pd

# Cargar los CSV correctamente desde la línea 9
pm10 = pd.read_csv("PM10_CLUB_UNION.csv", sep=";", skiprows=8)[['Fecha inicial', 'PM10']].rename(columns={'Fecha inicial': 'Fecha'})
pm25 = pd.read_csv("PM2.5_CLUB_UNION.csv", sep=";", skiprows=8)[['Fecha inicial', 'PM2.5']].rename(columns={'Fecha inicial': 'Fecha'})
so2 = pd.read_csv("SO2_CLUB_UNION.csv", sep=";", skiprows=8)[['Fecha inicial', 'SO2']].rename(columns={'Fecha inicial': 'Fecha'})
no2 = pd.read_csv("NO2_CLUB_UNION.csv", sep=";", skiprows=8)[['Fecha inicial', 'NO2']].rename(columns={'Fecha inicial': 'Fecha'})
o3  = pd.read_csv("O3_CLUB_UNION.csv",  sep=";", skiprows=8)[['Fecha inicial', 'O3']].rename(columns={'Fecha inicial': 'Fecha'})

# Unificar y limpiar
df = pm10.merge(pm25, on="Fecha").merge(so2, on="Fecha").merge(no2, on="Fecha").merge(o3, on="Fecha")
df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
df_clean = df.dropna().sort_values(by='Fecha')

# Crear variable objetivo
def clasificar_ica(valor):
    if valor <= 50:
        return "Buena"
    elif valor <= 100:
        return "Moderada"
    else:
        return "Mala"

df_clean['ICA_categoria'] = df_clean['PM10'].apply(clasificar_ica)


In [ ]:
# Entrenamiento y evaluación de 3 modelos supervisados

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Variables predictoras y objetivo
X = df_clean[['PM10', 'PM2.5', 'SO2', 'NO2', 'O3']]
y = df_clean['ICA_categoria']

# Codificar clases
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# División entrenamiento/prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Modelos
modelos = {
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": GaussianNB(),
    "Regresión Logística": LogisticRegression(max_iter=1000)
}

# Resultados
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)
    acc = accuracy_score(y_test, predicciones)
    matriz = confusion_matrix(y_test, predicciones)
    print(f"Modelo: {nombre}")
    print(f"Accuracy: {acc:.2f}")
    print("Matriz de Confusión:")
    print(matriz)
    print("-" * 30)


Modelo: Random Forest
Accuracy: 1.00
Matriz de Confusión:
[[164]]
------------------------------
Modelo: Naive Bayes
Accuracy: 1.00
Matriz de Confusión:
[[164]]
------------------------------
Modelo: Regresión Logística
Accuracy: 1.00
Matriz de Confusión:
[[164]]
------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


¿Qué es ICA_categoria?
Es una nueva columna que creamos para clasificar la calidad del aire en tres categorías:

Buena: cuando el valor de PM10 es menor o igual a 50

Moderada: cuando está entre 51 y 100

Mala: cuando es mayor a 100

Esto se llama feature engineering, porque transformamos un valor numérico (PM10) en una variable categórica que nos permite predecir el estado del aire.

¿Qué hicimos con los modelos?
Usamos 3 algoritmos de clasificación llamados modelos supervisados:

Random Forest

Naive Bayes

Regresión Logística

Estos modelos sirven para predecir automáticamente si la calidad del aire es buena, moderada o mala según los niveles de contaminantes (PM10, PM2.5, SO2, NO2, O3).

¿Cómo lo hicimos?
Limpieza de datos: quitamos valores nulos, corregimos formatos de fecha, y unificamos los archivos.

Análisis exploratorio: generamos gráficas de distribución y una matriz de correlación entre contaminantes.

Creamos la variable ICA_categoria basada en PM10.

Entrenamos 3 modelos para que aprendieran a clasificar la calidad del aire.

Evaluamos los modelos con métricas como accuracy y matriz de confusión.

Resultado:
¡Los 3 modelos clasificaron correctamente todos los casos!

El modelo ya puede ser usado para predecir la categoría del aire automáticamente si le damos nuevos valores de contaminantes.

En este proyecto se desarrolló un modelo predictivo para clasificar la calidad del aire en categorías (Buena, Moderada o Mala) usando datos reales de contaminación.
Se aplicaron técnicas de limpieza, análisis exploratorio, creación de variables (feature engineering) y modelado supervisado con tres algoritmos.
Como resultado, los modelos fueron entrenados exitosamente y están listos para ser usados en una aplicación real o educativa para la predicción del estado del aire.